# PROJECT 5 
Kehinde Ajayi

## CREATE MULTI-YEAR DATAFRAMES FROM FARS DATA

In [25]:
import numpy as np
import pandas as pd





In [2]:
# function for merging data

def get_merged_dataframes(main_df, main_df_columns, csv_files, year, merge_columns):
    
    df1 = pd.read_csv(f'./data/{year}_fars/{main_df}.csv', encoding='unicode_escape')
    df1.columns = df1.columns.str.lower()
    merged_dataframe = df1[df1.columns.intersection(main_df_columns)]
    
    for csv in csv_files:
        try:
            df2 = pd.read_csv(f'./data/{year}_fars/{csv}.csv')
            df2.columns = df2.columns.str.lower() 
            merged_dataframe = pd.merge(merged_dataframe, df2, how='outer', on=merge_columns, suffixes=('', '_remove'))
            merged_dataframe = merged_dataframe.drop([column for column in merged_dataframe.columns if 'remove' in column], axis=1)
        except FileNotFoundError:
            print(f'The {csv} file does not exist for the year {year}.')           
    
    merged_dataframe.loc[:,'data_year'] = year
    
    return merged_dataframe

#### Merge all person-level data

In [5]:
def combine_person_data(start_year, end_year=None):  
    
    person_columns = ['state', 'st_case', 'veh_no', 'per_no', 'county', 'age', 'sex', 
                      'per_typ', 'inj_sev', 'seat_pos', 'rest_use', 'rest_mis', 'air_bag', 'ejection',
                      'ej_path', 'extricat', 'drinking', 'alc_det', 'alc_status', 'atst_typ', 'alc_res',
                      'drugs', 'drug_det', 'dstatus', 'hospital', 'doa', 'death_da', 'death_mo', 
                      'death_yr', 'death_hr', 'death_mn', 'death_tm', 'n_mot_no', 'location',
                      'helm_use', 'helm_mis', 'str_veh']
    
    csvs_to_merge = ['per_aux', 'nmprior', 'nmcrash', 'nmimpair',
                       'safetyeq', 'nmdistract', 'drugs']
    
    merge_columns = ['st_case', 'veh_no', 'per_no']
    
    if end_year is None:
        end_year = start_year
    
    full_df_list = [get_merged_dataframes('person', person_columns, csvs_to_merge, year, merge_columns) for year in range(start_year, end_year + 1)]
    
    return pd.concat(full_df_list, ignore_index=True)
        

In [16]:
      
full_person_df = combine_person_data(2010, 2020)


The nmdistract file does not exist for the year 2010.
The drugs file does not exist for the year 2010.


/tmp/ipykernel_328328/1400665811.py:18: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df_list = [get_merged_dataframes('person', person_columns, csvs_to_merge, year, merge_columns) for year in range(start_year, end_year + 1)]


The nmdistract file does not exist for the year 2011.
The drugs file does not exist for the year 2011.
The nmdistract file does not exist for the year 2012.
The drugs file does not exist for the year 2012.
The nmdistract file does not exist for the year 2013.
The drugs file does not exist for the year 2013.
The nmdistract file does not exist for the year 2014.
The drugs file does not exist for the year 2014.


/tmp/ipykernel_328328/1400665811.py:18: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df_list = [get_merged_dataframes('person', person_columns, csvs_to_merge, year, merge_columns) for year in range(start_year, end_year + 1)]


The nmdistract file does not exist for the year 2015.
The drugs file does not exist for the year 2015.
The nmdistract file does not exist for the year 2016.
The drugs file does not exist for the year 2016.
The nmdistract file does not exist for the year 2017.
The drugs file does not exist for the year 2017.
The nmdistract file does not exist for the year 2018.


In [17]:
full_person_df.shape

(962352, 92)

In [18]:
full_person_df.head()

,state,st_case,veh_no,per_no,n_mot_no,county,age,sex,per_typ,inj_sev,seat_pos,rest_use,rest_mis,air_bag,ejection,ej_path,extricat,drinking,alc_det,alc_status,atst_typ,alc_res,drugs,drug_det,dstatus,hospital,doa,death_da,death_mo,death_yr,death_hr,death_mn,death_tm,location,a_age1,a_age2,a_age3,a_age4,a_age5,a_age6,a_age7,a_age8,a_age9,year,a_ptype,a_restuse,a_helmuse,a_alctes,a_hisp,a_rcat,a_hrace,a_eject,a_perinj,a_loc,a_doa,mpr_act,mtm_crsh,nmimpair,msafeqmt,data_year,str_veh,statename,mpr_actname,mtm_crshname,nmimpairname,msafeqmtname,nmhelmet,nmhelmetname,nmpropad,nmpropadname,nmothpro,nmothproname,nmrefclo,nmrefcloname,nmlight,nmlightname,nmothpre,nmothprename,drugspec,drugspecname,drugres,drugresname,helm_use,helm_mis,mnmdstrd,mnmdstrdname,nmaction,nmactionname,nmcc,nmccname,nmdistract,nmdistractname
0,1,10001,1,1,0.0,81,51,2,1,4,11,7,0,99,0,0,0,9,9,2,1,11,9,8,2,0,7,15,1,2010,4,10,410,0,3,5,9,6,6,6,8,5,2,2010,1,2,2,2,1,2,3,1,1,1,1,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10001,1,2,0.0,81,999,1,2,3,13,7,0,99,1,9,0,8,9,0,0,96,8,8,0,1,0,88,88,8888,88,88,8888,0,5,6,13,8,10,10,12,7,3,2010,2,2,2,3,0,0,0,2,6,1,5,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10002,1,1,0.0,35,44,2,1,4,11,7,0,20,0,0,0,0,9,2,1,0,0,8,2,5,0,11,1,2010,6,35,635,0,3,5,8,5,5,5,7,4,2,2010,1,2,2,1,1,2,3,1,1,1,3,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10003,1,1,0.0,97,27,1,1,4,11,5,0,0,8,0,0,9,9,2,1,0,9,8,2,5,0,14,1,2010,15,56,1556,0,3,4,7,4,4,4,6,2,2,2010,1,2,1,1,1,1,2,1,1,1,3,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10003,2,1,0.0,97,45,2,1,0,11,3,0,8,0,0,0,0,5,0,0,96,0,8,0,0,0,88,88,8888,88,88,8888,0,3,5,9,6,6,6,8,4,2,2010,1,1,2,3,0,0,0,1,6,1,5,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Merge all driver-level data

In [19]:
def combine_driver_data(start_year, end_year=None):  
    
    driver_columns = ['state', 'st_case', 'veh_no', 'dr_pres', 'l_state', 'dr_zip', 'l_type', 'l_status', 
                      'cdl_stat', 'l_endors', 'l_compl', 'l_restri', 'dr_hgt', 'dr_wgt', 'prev_oth',
                      'first_mo', 'first_yr', 'last_mo', 'last_yr', 'speedrel' ]
          
    csvs_to_merge = ['veh_aux', 'violatn', 'drimpair', 'driverrf'] 
    
    merge_columns = ['st_case', 'veh_no']
    
    if end_year is None:
        end_year = start_year
    
    full_df_list = [get_merged_dataframes('vehicle', driver_columns, csvs_to_merge, year, merge_columns) for year in range(start_year, end_year + 1)]
    
    return pd.concat(full_df_list, ignore_index=True)
        

In [20]:
      
full_driver_df = combine_driver_data(2010, 2020)


/tmp/ipykernel_328328/350924323.py:14: DtypeWarning: Columns (49,50) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df_list = [get_merged_dataframes('vehicle', driver_columns, csvs_to_merge, year, merge_columns) for year in range(start_year, end_year + 1)]


The driverrf file does not exist for the year 2010.


/tmp/ipykernel_328328/350924323.py:14: DtypeWarning: Columns (48,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df_list = [get_merged_dataframes('vehicle', driver_columns, csvs_to_merge, year, merge_columns) for year in range(start_year, end_year + 1)]


The driverrf file does not exist for the year 2011.


/tmp/ipykernel_328328/350924323.py:14: DtypeWarning: Columns (60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df_list = [get_merged_dataframes('vehicle', driver_columns, csvs_to_merge, year, merge_columns) for year in range(start_year, end_year + 1)]


...

/tmp/ipykernel_328328/350924323.py:14: DtypeWarning: Columns (14,56,58,193,195) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df_list = [get_merged_dataframes('vehicle', driver_columns, csvs_to_merge, year, merge_columns) for year in range(start_year, end_year + 1)]


The driverrf file does not exist for the year 2018.


/tmp/ipykernel_328328/350924323.py:14: DtypeWarning: Columns (56,58,193,195) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df_list = [get_merged_dataframes('vehicle', driver_columns, csvs_to_merge, year, merge_columns) for year in range(start_year, end_year + 1)]


The driverrf file does not exist for the year 2019.


/tmp/ipykernel_328328/350924323.py:14: DtypeWarning: Columns (14,56,58,179,181) have mixed types.Specify dtype option on import or set low_memory=False.
  full_df_list = [get_merged_dataframes('vehicle', driver_columns, csvs_to_merge, year, merge_columns) for year in range(start_year, end_year + 1)]


In [21]:
full_driver_df.shape

(584270, 46)

In [22]:
full_driver_df.head()

,state,st_case,veh_no,dr_pres,l_state,dr_zip,l_status,l_type,cdl_stat,l_endors,l_compl,l_restri,dr_hgt,dr_wgt,prev_oth,first_mo,first_yr,last_mo,last_yr,speedrel,year,a_drdis,a_drdro,a_vrd,a_body,a_imp1,a_imp2,a_vroll,a_lic_s,a_lic_c,a_cdl_s,a_mc_l_s,a_spveh,a_sbus,a_mod_yr,a_fire_exp,mviolatn,drimpair,data_year,statename,mviolatnname,drimpairname,violation,violationname,driverrf,driverrfname
0,1.0,10001,1,1.0,13.0,30296.0,6.0,1.0,0.0,0.0,3.0,1.0,65.0,180.0,0.0,9.0,2008.0,11.0,2008.0,0.0,2010.0,2.0,1.0,1.0,3.0,7.0,7.0,1.0,1.0,1.0,2.0,4.0,2.0,3.0,2000.0,2.0,0.0,2,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,10002,1,1.0,1.0,36401.0,6.0,1.0,0.0,0.0,3.0,0.0,65.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,2010.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,4.0,2.0,3.0,1998.0,1.0,0.0,0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,10003,1,1.0,1.0,36607.0,6.0,1.0,0.0,0.0,2.0,1.0,80.0,300.0,2.0,9.0,2009.0,10.0,2009.0,1.0,2010.0,2.0,2.0,2.0,7.0,6.0,6.0,2.0,4.0,2.0,2.0,2.0,1.0,3.0,2005.0,1.0,0.0,0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,10003,2,1.0,1.0,36606.0,6.0,1.0,0.0,0.0,3.0,0.0,61.0,140.0,0.0,8.0,2008.0,8.0,2008.0,0.0,2010.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,4.0,2.0,3.0,2008.0,1.0,0.0,0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,10003,3,1.0,1.0,36607.0,6.0,1.0,0.0,0.0,3.0,1.0,66.0,124.0,0.0,0.0,0.0,0.0,0.0,0.0,2010.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,4.0,2.0,3.0,2006.0,1.0,0.0,0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save multi-year person- and driver-level data in separate .csv files

In [23]:
full_person_df.to_csv('./data/person-level_data_2010-2020.csv', index=False)

In [24]:
full_driver_df.to_csv('./data/driver-level_data_2010-2020.csv', index=False)